## Diccionario de Datos

### Descripción General de los Datos
El conjunto de datos `BD_creditos.csv` contiene información detallada sobre créditos otorgados. El objetivo es analizar estos datos para comprender las características de los préstamos y los perfiles de los clientes, lo que podría servir para evaluar el riesgo crediticio.

### Caracterización de los Datos

A continuación se detalla el tipo y la descripción de cada variable presente en el dataset:

| Variable | Tipo de Dato | Subtipo | Descripción |
| :--- | :--- | :--- | :--- |
| **tipo_credito** | Categórico | Nominal | Modalidad o tipo del crédito otorgado. |
| **fecha_prestamo** | Fecha/Hora | Continuo | Fecha en la que se desembolsó el préstamo. |
| **capital_prestado** | Numérico | Continuo | Monto total del capital prestado al cliente. |
| **plazo_meses** | Numérico | Discreto | Número de meses acordado para pagar el crédito. |
| **edad_cliente** | Numérico | Discreto | Edad del cliente al momento de solicitar el crédito. |
| **tipo_laboral** | Categórico | Nominal | Condición laboral del cliente (ej. empleado, independiente). |
| **salario_cliente** | Numérico | Continuo | Ingreso salarial reportado por el cliente. |
| **total_otros_prestamos**| Numérico | Continuo | Suma de las deudas en otras entidades. |
| **cuota_pactada** | Numérico | Continuo | Valor de la cuota mensual que el cliente se comprometió a pagar. |
| **puntaje** | Numérico | Discreto | Calificación o puntaje de riesgo interno asignado por la entidad. |
| **puntaje_datacredito**| Numérico | Discreto | Puntaje crediticio del cliente según la central de riesgo DataCrédito. |
| **cant_creditosvigentes**| Numérico | Discreto | Número de créditos que el cliente tiene activos actualmente. |
| **huella_consulta** | Numérico | Discreto | Cantidad de veces que el historial crediticio del cliente ha sido consultado. |
| **saldo_mora** | Numérico | Continuo | Monto de la deuda que se encuentra en mora. |
| **saldo_total** | Numérico | Continuo | Deuda total del cliente, incluyendo capital e intereses. |
| **saldo_principal** | Numérico | Continuo | Saldo pendiente correspondiente solo al capital del préstamo. |
| **saldo_mora_codeudor** | Numérico | Continuo | Monto de la deuda en mora correspondiente al codeudor. |
| **creditos_sectorFinanciero**| Numérico | Discreto | Número de créditos que el cliente tiene en el sector financiero. |
| **creditos_sectorCooperativo**| Numérico | Discreto | Número de créditos que el cliente tiene en el sector cooperativo. |
| **creditos_sectorReal** | Numérico | Discreto | Número de créditos con empresas del sector real (ej. almacenes). |
| **promedio_ingresos_datacredito**| Numérico | Continuo | Ingreso promedio del cliente reportado en DataCrédito. |
| **tendencia_ingresos** | Categórico | Ordinal | Tendencia de los ingresos del cliente (ej. aumenta, disminuye, estable). |
| **Pago_atiempo** | Categórico | Nominal (Dicotómico) | Variable objetivo. Indica si el cliente ha pagado a tiempo ('SI', 'NO'). |

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('ggplot')
%matplotlib inline

In [3]:
df = pd.read_csv('BD_creditos.csv')

In [4]:
# Calcular valores nulos y su porcentaje
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Crear un DataFrame para mostrar los resultados
missing_df = pd.DataFrame({
    'Columna': missing_values.index,
    'Valores Nulos': missing_values.values,
    'Porcentaje (%)': missing_percentage.values
})

# Mostrar solo las columnas que tienen valores nulos
print("REVISIÓN DE VALORES NULOS")
display(missing_df[missing_df['Valores Nulos'] > 0].sort_values(by='Porcentaje (%)', ascending=False))

if missing_df['Valores Nulos'].sum() == 0:
    print("\n✅ No se encontraron valores nulos en el dataset.")

REVISIÓN DE VALORES NULOS


,Columna,Valores Nulos,Porcentaje (%)
21,tendencia_ingresos,2932,27.241475
20,promedio_ingresos_datacredito,2930,27.222893
16,saldo_mora_codeudor,590,5.481743
15,saldo_principal,405,3.762891
13,saldo_mora,156,1.449410
14,saldo_total,156,1.449410
10,puntaje_datacredito,6,0.055747


### Revisión de Nulos
A continuación, se realiza un análisis cuantitativo para identificar la cantidad y el porcentaje de valores nulos en cada columna del dataset. Esto es fundamental para decidir las estrategias de limpieza y preprocesamiento.

In [5]:
print("Primeras 5 filas:")
display(df.head())
print("\nEstadísticas descriptivas:")
display(df.describe(include='all'))

Primeras 5 filas:


,tipo_credito,fecha_prestamo,capital_prestado,plazo_meses,edad_cliente,tipo_laboral,salario_cliente,total_otros_prestamos,cuota_pactada,puntaje,...,saldo_mora,saldo_total,saldo_principal,saldo_mora_codeudor,creditos_sectorFinanciero,creditos_sectorCooperativo,creditos_sectorReal,promedio_ingresos_datacredito,tendencia_ingresos,Pago_atiempo
0,7,12/21/2024 11:31,3692160,10,42,Independiente,8000000,2500000,341296,88.768094,...,0.0,51258.0,51258.0,0.0,5,0,0,908526.0,Estable,1
1,4,4/22/2025 9:47,840000,6,60,Empleado,3000000,2000000,124876,95.227787,...,0.0,8673.0,8673.0,0.0,0,0,2,939017.0,Creciente,1
2,9,1/8/2026 12:22,5974028,10,36,Independiente,4036000,829000,529554,47.613894,...,0.0,18702.0,18702.0,0.0,3,0,0,NaN,NaN,0
3,4,8/4/2025 12:04,1671240,6,48,Empleado,1524547,498000,252420,95.227787,...,0.0,15782.0,15782.0,0.0,3,0,0,1536193.0,Creciente,1
4,9,4/26/2025 11:24,2781636,11,44,Empleado,5000000,4000000,217037,95.227787,...,0.0,204804.0,204804.0,0.0,3,0,1,933473.0,Creciente,1



Estadísticas descriptivas:


,tipo_credito,fecha_prestamo,capital_prestado,plazo_meses,edad_cliente,tipo_laboral,salario_cliente,total_otros_prestamos,cuota_pactada,puntaje,...,saldo_mora,saldo_total,saldo_principal,saldo_mora_codeudor,creditos_sectorFinanciero,creditos_sectorCooperativo,creditos_sectorReal,promedio_ingresos_datacredito,tendencia_ingresos,Pago_atiempo
count,10763.000000,10763,1.076300e+04,10763.000000,10763.000000,10763,1.076300e+04,1.076300e+04,1.076300e+04,10763.000000,...,10607.000000,1.060700e+04,1.035800e+04,10173.000000,10763.000000,10763.000000,10763.000000,7.833000e+03,7831,10763.000000
unique,NaN,10444,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,NaN
top,NaN,11/28/2024 7:46,NaN,NaN,NaN,Empleado,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Creciente,NaN
freq,NaN,4,NaN,NaN,NaN,6754,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5294,NaN
mean,5.411131,NaN,2.434315e+06,10.575583,43.948620,NaN,1.721643e+07,6.238870e+06,2.436174e+05,91.170036,...,7.746017,4.593741e+04,4.034617e+04,0.260002,2.779987,0.269813,1.302704,2.005157e+06,NaN,0.952523
std,2.338279,NaN,1.909643e+06,6.632082,15.060877,NaN,3.554767e+08,1.184183e+08,2.104937e+05,16.465441,...,225.955117,1.062698e+05,7.124244e+04,21.772917,2.748807,0.716471,1.824430,2.144116e+06,NaN,0.212668
min,4.000000,NaN,3.600000e+05,2.000000,19.000000,NaN,0.000000e+00,0.000000e+00,2.394400e+04,-38.009990,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,NaN,0.000000
25%,4.000000,NaN,1.224831e+06,6.000000,33.000000,NaN,2.000000e+06,5.000000e+05,1.210415e+05,95.227787,...,0.000000,2.898000e+03,2.690000e+03,0.000000,1.000000,0.000000,0.000000,9.251570e+05,NaN,1.000000
50%,4.000000,NaN,1.921920e+06,10.000000,42.000000,NaN,3.000000e+06,1.000000e+06,1.828630e+05,95.227787,...,0.000000,1.617800e+04,1.444250e+04,0.000000,2.000000,0.000000,1.000000,1.204496e+06,NaN,1.000000
75%,9.000000,NaN,3.084840e+06,12.000000,53.000000,NaN,4.875808e+06,2.000000e+06,2.878335e+05,95.227787,...,0.000000,5.298200e+04,4.763225e+04,0.000000,4.000000,0.000000,2.000000,2.231859e+06,NaN,1.000000


In [6]:
# Análisis de variables irrelevantes para predicción de pago a tiempo
print("=== ANÁLISIS DE VARIABLES IRRELEVANTES ===")

# 1. Análisis de correlación con la variable objetivo
correlations_with_target = df.select_dtypes(include=[np.number]).corr()['Pago_atiempo'].abs().sort_values(ascending=False)
print("\nCorrelaciones con variable objetivo (Pago_atiempo):")
print(correlations_with_target)

# 2. Identificar variables con varianza muy baja o cero
print("\n=== ANÁLISIS DE VARIANZA ===")
numerical_cols = df.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if col != 'Pago_atiempo':
        variance = df[col].var()
        unique_values = df[col].nunique()
        print(f"{col}: Varianza={variance:.4f}, Valores únicos={unique_values}")

# 3. Variables que podrían ser irrelevantes o problemáticas
variables_irrelevantes = {
    'fecha_prestamo': 'Fecha específica no es predictiva por sí misma (usar características derivadas)',
    'saldo_mora': 'Puede causar data leakage - refleja comportamiento de pago ya ocurrido',
    'saldo_total': 'Puede causar data leakage - información posterior al evento',
    'saldo_principal': 'Puede causar data leakage - información posterior al evento',
    'saldo_mora_codeudor': 'Puede causar data leakage - información posterior al evento'
}

print("\n=== VARIABLES POTENCIALMENTE IRRELEVANTES O PROBLEMÁTICAS ===")
for var, razon in variables_irrelevantes.items():
    if var in df.columns:
        print(f"❌ {var}: {razon}")

# 4. Variables con muchos valores faltantes
print("\n=== VARIABLES CON MUCHOS VALORES FALTANTES ===")
missing_threshold = 0.3  # 30% de valores faltantes
for col in df.columns:
    missing_pct = df[col].isnull().sum() / len(df)
    if missing_pct > missing_threshold:
        print(f"⚠️ {col}: {missing_pct:.2%} valores faltantes")

# 5. Variables identificadores (no predictivas)
identificadores = ['CustomerID'] if 'CustomerID' in df.columns else []
if identificadores:
    print(f"\n=== VARIABLES IDENTIFICADORAS (NO PREDICTIVAS) ===")
    for var in identificadores:
        print(f"🔍 {var}: Variable identificadora, no aporta valor predictivo")

=== ANÁLISIS DE VARIABLES IRRELEVANTES ===

Correlaciones con variable objetivo (Pago_atiempo):
Pago_atiempo                     1.000000
puntaje                          0.923134
huella_consulta                  0.073737
saldo_mora                       0.073458
puntaje_datacredito              0.067882
plazo_meses                      0.063105
capital_prestado                 0.040624
promedio_ingresos_datacredito    0.039867
edad_cliente                     0.032252
creditos_sectorReal              0.023306
creditos_sectorFinanciero        0.021390
creditos_sectorCooperativo       0.021267
saldo_total                      0.014364
cuota_pactada                    0.011814
saldo_principal                  0.011473
total_otros_prestamos            0.010041
cant_creditosvigentes            0.008829
salario_cliente                  0.003981
saldo_mora_codeudor              0.002631
tipo_credito                     0.000951
Name: Pago_atiempo, dtype: float64

=== ANÁLISIS DE VARIANZA ===

In [7]:
# Eliminar variables irrelevantes identificadas
print("=== ELIMINACIÓN DE VARIABLES IRRELEVANTES ===")

# Variables a eliminar basadas en el análisis
variables_a_eliminar = [
    'fecha_prestamo',  # Fecha específica - usaremos características derivadas
    'saldo_mora',      # Data leakage potencial
    'saldo_total',     # Data leakage potencial  
    'saldo_principal', # Data leakage potencial
    'saldo_mora_codeudor'  # Data leakage potencial
]

# Verificar que las variables existen antes de eliminar
variables_existentes = [var for var in variables_a_eliminar if var in df.columns]
variables_no_encontradas = [var for var in variables_a_eliminar if var not in df.columns]

print(f"Variables a eliminar encontradas: {variables_existentes}")
if variables_no_encontradas:
    print(f"Variables no encontradas en el dataset: {variables_no_encontradas}")

# Crear dataset sin variables irrelevantes
df_clean = df.drop(columns=variables_existentes)

print(f"\nDimensiones originales: {df.shape}")
print(f"Dimensiones después de eliminar variables irrelevantes: {df_clean.shape}")
print(f"Variables eliminadas: {len(variables_existentes)}")

# Mostrar las variables que quedan
print(f"\nVariables restantes ({df_clean.shape[1]}):")
for i, col in enumerate(df_clean.columns, 1):
    print(f"{i:2d}. {col}")

=== ELIMINACIÓN DE VARIABLES IRRELEVANTES ===
Variables a eliminar encontradas: ['fecha_prestamo', 'saldo_mora', 'saldo_total', 'saldo_principal', 'saldo_mora_codeudor']

Dimensiones originales: (10763, 23)
Dimensiones después de eliminar variables irrelevantes: (10763, 18)
Variables eliminadas: 5

Variables restantes (18):
 1. tipo_credito
 2. capital_prestado
 3. plazo_meses
 4. edad_cliente
 5. tipo_laboral
 6. salario_cliente
 7. total_otros_prestamos
 8. cuota_pactada
 9. puntaje
10. puntaje_datacredito
11. cant_creditosvigentes
12. huella_consulta
13. creditos_sectorFinanciero
14. creditos_sectorCooperativo
15. creditos_sectorReal
16. promedio_ingresos_datacredito
17. tendencia_ingresos
18. Pago_atiempo
